In [ ]:
pip install category_encoders

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 4.6 MB/s eta 0:00:00


In [ ]:
# Chargement des données optimisé
print("Chargement des données...")
import pandas as pd
import numpy as np
from category_encoders import CountEncoder
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.impute import SimpleImputer

# Charger depuis CSV (beaucoup plus rapide que Excel)
X_train = pd.read_csv('train_input_Z61KlZo.csv')
y_train = pd.read_csv('train_output_DzPxaPY.csv')

# Réduction de l'utilisation mémoire
for col in X_train.select_dtypes(include=['float']).columns:
    X_train[col] = pd.to_numeric(X_train[col], downcast='float')
for col in X_train.select_dtypes(include=['int']).columns:
    X_train[col] = pd.to_numeric(X_train[col], downcast='integer')
for col in X_train.select_dtypes(include='object').columns:
    if X_train[col].nunique() / len(X_train[col]) < 0.5:
        X_train[col] = X_train[col].astype('category')

# Afficher un aperçu des données pour vérifier leur chargement
print("Aperçu de X_train :")
display(X_train.head())
print("Aperçu de y_train :")
display(y_train.head())

# Traitement des valeurs manquantes dans les colonnes numériques
print("Traitement des valeurs manquantes dans les colonnes numériques...")

numeric_columns = X_train.select_dtypes(include=['number']).columns
num_imputer = SimpleImputer(strategy='constant', fill_value=0)
X_train[numeric_columns] = num_imputer.fit_transform(X_train[numeric_columns])

# Identifier les colonnes non numériques
fill_cols = [item for item in X_train.columns if item not in numeric_columns]
non_numeric_columns = X_train[fill_cols].select_dtypes(exclude=['number']).columns

# Remplir les NaN des colonnes non numériques avec 'missing'
cat_imputer = SimpleImputer(strategy='constant', fill_value='missing')
X_train[non_numeric_columns] = cat_imputer.fit_transform(X_train[non_numeric_columns])

print("Traitement des valeurs manquantes terminé.")

# Préparation des données pour l'entraînement
print("Préparation des données pour l'entraînement...")

# Suppression des colonnes inutiles
X_train = X_train.drop(['ID', 'ANNEE_ASSURANCE'], axis=1)

# Encodage des variables catégoriques avec CountEncoder
encoder = CountEncoder(cols=non_numeric_columns)
encoder.fit(X_train)
X_train_enc = encoder.transform(X_train)

print("Préparation terminée.")




Chargement des données...


<ipython-input-3-eea2ae1e9201>:11: DtypeWarning: Columns (16,17,29,30,31,126,128,129,132,133,135,138,371) have mixed types. Specify dtype option on import or set low_memory=False.
  X_train = pd.read_csv('train_input_Z61KlZo.csv')


Aperçu de X_train :


,ID,ACTIVIT2,VOCATION,TYPERS,ANCIENNETE,ADOSS,CARACT1,CARACT2,CARACT3,INDEM1,...,NBJRR100_MMAX_A,NBJRR100_MSOM_A,RR_VOR_MM_A,RR_VOR_MMAX_A,RRAB_VOR_MM_A,RRAB_VOR_MMAX_A,ANNEE_ASSURANCE,ESPINSEE,AN_EXERC,ZONE
0,1,ACT1,VOC6,1,0,N,N,NaN,NaN,N,...,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,NaN,ANNEE5,3
1,2,ACT1,VOC6,1,0,N,N,NaN,NaN,N,...,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,NaN,ANNEE5,3
2,3,ACT1,VOC6,1,2,N,R,NaN,NaN,N,...,NaN,NaN,NaN,NaN,NaN,NaN,0.402740,NaN,ANNEE5,3
3,4,ACT1,VOC6,2,0,N,N,NaN,NaN,N,...,NaN,NaN,NaN,NaN,NaN,NaN,0.246575,NaN,ANNEE5,3
4,5,ACT1,VOC6,2,1,N,N,NaN,NaN,N,...,01. <= 0,01. <= 0,03. <= 102,03. <= 232,04. >= 25,02. <= 57,0.838356,NaN,ANNEE5,3


Aperçu de y_train :


,ID,FREQ,CM,ANNEE_ASSURANCE,CHARGE
0,1,0.0,0.0,1.000000,0.0
1,2,0.0,0.0,1.000000,0.0
2,3,0.0,0.0,0.402740,0.0
3,4,0.0,0.0,0.246575,0.0
4,5,0.0,0.0,0.838356,0.0


Traitement des valeurs manquantes dans les colonnes numériques...
Traitement des valeurs manquantes terminé.
Préparation des données pour l'entraînement...
Préparation terminée.


In [ ]:
# Entraîner les modèles
print("Entraînement des modèles")

# Modèle pour prédire 'FREQ'
rf_freq = RandomForestRegressor(n_estimators=50, max_depth=10, random_state=42)
rf_freq.fit(X_train_enc, y_train['FREQ'])
print("Modèle pour 'FREQ' entraîné avec succès.")

# Prédire 'FREQ' sur l'ensemble d'entraînement
y_train_pred_freq = rf_freq.predict(X_train_enc)



Entraînement des modèles
Modèle pour 'FREQ' entraîné avec succès.


In [ ]:
# Modèle pour prédire 'CM'
gb_cm = GradientBoostingRegressor(n_estimators=50, max_depth=5, random_state=42)
gb_cm.fit(X_train_enc, y_train['CM'])
print("Modèle pour 'CM' entraîné avec succès.")

# Prédire 'CM' sur l'ensemble d'entraînement
y_train_pred_cm = gb_cm.predict(X_train_enc)



Modèle pour 'CM' entraîné avec succès.


In [ ]:
# Calculer la prédiction combinée pour 'CHARGE'
y_train_pred = y_train_pred_freq * y_train_pred_cm * y_train['ANNEE_ASSURANCE']

# Calculer le RMSE sur l'ensemble d'entraînement
rmse = np.sqrt(mean_squared_error(y_train['CHARGE'], y_train_pred))
print(f"RMSE sur l'ensemble d'entraînement : {rmse:.2f}")

RMSE sur l'ensemble d'entraînement : 6746.26


In [ ]:
#Save Models

In [ ]:
import joblib  # Pour sauvegarder les modèles

# Sauvegarder les modèles
print("Sauvegarde des modèles...")
joblib.dump(rf_freq, 'rf_freq_model.pkl')
joblib.dump(gb_cm, 'gb_cm_model.pkl')
joblib.dump(encoder, 'encoder.pkl')
print("Modèles sauvegardés avec succès.")

Sauvegarde des modèles...
Modèles sauvegardés avec succès.


In [ ]:
#Testing different encoding techniques

In [ ]:
from category_encoders import CountEncoder, TargetEncoder, CatBoostEncoder

In [ ]:
# Tester différents encodeurs
print("\nComparaison de différents encodeurs...")
encoders = {
    'CountEncoder': CountEncoder(cols=non_numeric_columns),
    'TargetEncoder': TargetEncoder(cols=non_numeric_columns),
    'CatBoostEncoder': CatBoostEncoder(cols=non_numeric_columns)
}

rmse_results = {}

for name, encoder_variant in encoders.items():
    print(f"\n--- Test de {name} ---")
    encoder_variant.fit(X_train, y_train['CHARGE'])
    X_train_enc_variant = encoder_variant.transform(X_train)

    rf_freq_variant = RandomForestRegressor(n_estimators=50, max_depth=10, random_state=42)
    rf_freq_variant.fit(X_train_enc_variant, y_train['FREQ'])
    y_train_pred_freq_variant = rf_freq_variant.predict(X_train_enc_variant)

    gb_cm_variant = GradientBoostingRegressor(n_estimators=50, max_depth=5, random_state=42)
    gb_cm_variant.fit(X_train_enc_variant, y_train['CM'])
    y_train_pred_cm_variant = gb_cm_variant.predict(X_train_enc_variant)

    y_train_pred_variant = y_train_pred_freq_variant * y_train_pred_cm_variant * y_train['ANNEE_ASSURANCE']

    rmse_variant = np.sqrt(mean_squared_error(y_train['CHARGE'], y_train_pred_variant))
    rmse_results[name] = rmse_variant
    print(f"RMSE pour {name} : {rmse_variant:.2f}")

print("\nRésumé des RMSE pour chaque encodeur :")
for name, score in rmse_results.items():
    print(f"{name} : {score:.2f}")


Comparaison de différents encodeurs...

--- Test de CountEncoder ---
RMSE pour CountEncoder : 6746.26

--- Test de TargetEncoder ---
RMSE pour TargetEncoder : 6725.96

--- Test de CatBoostEncoder ---
RMSE pour CatBoostEncoder : 6723.58

Résumé des RMSE pour chaque encodeur :
CountEncoder : 6746.26
TargetEncoder : 6725.96
CatBoostEncoder : 6723.58


In [ ]:
#Save the last trained model --> CatBoostEncoder

joblib.dump(rf_freq_variant, f'rf_freq_model_{name}.pkl')
joblib.dump(gb_cm_variant, f'gb_cm_model_{name}.pkl')
joblib.dump(encoder_variant, f'encoder_{name}.pkl')

print("Modèles et encodeur pour", name, "sauvegardés avec succès.")